# Privacy - attack model - part2 : Optimisation

reference for the WGAN : https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/

# Imports and Data

In [ ]:
%load_ext autoreload
%autoreload 2

from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
import pandas as pd

from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import backend
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Add
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.initializers import RandomUniform

from tensorflow.keras.datasets import mnist
from matplotlib import pyplot
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import datetime

os.sys.path.append("./src")
from utils import plot_img
from utils import load_mnist_data
from utils import pick_and_show_image
from models import define_critic

from models import define_critic
from models import generate_latent_points

from models import ClipConstraint

import codecs, json

# 1. Load models

## 1.1 load critic model

As the critic model is customed we build up a new model with the same architecture as the critic, load the weights of the one saved and load them into the new one.

In [ ]:
def load_critic_model(model_id):
    if 'attack_critic_model' in locals():
        del attack_critic_model
    attack_critic_model = define_critic()
    attack_critic_model.load_weights("model/attack_critic_model_weights_" + str(model_id) + ".h5")
    attack_critic_model._name = "attack_critic_model"
    print("Loading of the critic : {}".format(attack_critic_model.name))
    
    return attack_critic_model

## 1.2 load the gan

We use two gan with different names so that the model do not have the same layers' name when combined in the optimization model

In [ ]:
def load_gan_model(model_id):
    
    if 'attack_gan_model_1' in locals():
        del attack_gan_model_1
    attack_gan_model_1 = load_model("model/attack_gan_model_" + str(model_id) + ".h5")
    attack_gan_model_1._name = "attack_gan_model_1"
    print(attack_gan_model_1.name)

    if 'attack_gan_model_2' in locals():
        del attack_gan_model_2
    attack_gan_model_2 = load_model("model/attack_gan_model_" + str(model_id) + ".h5")
    attack_gan_model_2._name = "attack_gan_model_2"
    print(attack_gan_model_2.name)
    
    return attack_gan_model_1, attack_gan_model_2

# 2. Define optimization process

## 2.1 Define the model

We design a model which intermediate layer is of the dimension of the latent space. We want to generate a point in the latent space (__x__) that makes the generator produce a credible image (hand written-digit-like) which will be classified by the target model as the target label.
The outputs of the model are chosen to build a loss designed to perform this optimization program :
- __attack_critic_model(attack_gan_model_1(x))__ which the critic of the image generated by the GAN, the more the image is credible, the higher will be this output ;
- __target_model(attack_gan_model_2(x))__ which the class predicted by the target model, this one will be used to compute a difference with the label targetted

In [ ]:
def create_new_model_optim(target_model,
                           attack_critic_model, 
                           attack_gan_model_1, 
                           attack_gan_model_2,
                           random_init_dim=1):
    
    if 'gen_latent_values' in locals():
        del gen_latent_values
    
    if 'gen_attack_img' in locals():
        del gen_attack_img
    
    # generate a point in the latent space
    main_input = Input(shape = (random_init_dim, ), name='main_input')
    x = Dense(50,
              activation='linear',
#              kernel_initializer = RandomNormal(mean=0.0, stddev=1, seed=None),
              kernel_initializer = RandomUniform(-2, 2, seed=None),
              kernel_constraint = ClipConstraint(2),
              use_bias=False ,
              name = "gen_attack_img")(main_input)
    gen_latent_values = Model(main_input, x)

    # build up the model with the outputs that will be used to compute the loss
    model = Model(inputs=main_input, outputs = [attack_critic_model(attack_gan_model_1(x)),
                                                target_model(attack_gan_model_2(x))])
    return model, gen_latent_values

Only one layer should be trainable to perform the optimization process. If not we will retrain our models.

In [ ]:
def freeze_not_optim_layers(model):
    for layer in model.layers:
        if layer.name == "gen_attack_img":
            layer.trainable=True
        else:
            layer.trainable=False
        # print("layer : {}, trainable {}".format(layer.name, layer.trainable))
    print(model.summary())
    return model

## 2.2 Define the loss

The loss is compound of two parts :
- the credibility : is the image reconstructed a credible hand written digit ?
- the class proximity : is the class of the reconstructed image, the one targetted ?

In [ ]:
def loss_optim_fn(target, lambda_):
    def loss_optim(y_true, y_pred):
        # target.shape -> (5,)
        # y_pred is a list with
        # y_pred[0].shape -> (500, 1)
        # y_pred[1].shape -> (500, 5)
        # credibility part : this part caracterize the credibility of the reconstructed image in the space of the public data
        credibility = backend.mean(y_pred[0], axis=0)
        # credibility.shape -> (1,)
        
        # proximity part : define the absolute difference between the predicted for the reconstructed image and the class targetted
        # backend.mean(y_pred[1], axis=0).shape -> (5,)
        proximity_to_target = backend.mean(y_pred[1], axis=0) - target
        proximity_to_target = backend.abs(proximity_to_target)
        proximity_to_target = backend.mean(proximity_to_target)/2
        return backend.sum([credibility, lambda_*backend.log(proximity_to_target)], axis=-1)
    return loss_optim

# 3. Launch the whole process

## 3.1 Define the optimization loop

The optimization is repeated 5 times. The trial with best results is chosen.

In [ ]:
def init_and_fit_optim_model(x, y, target_model, n_epochs, model_id, loss, optimizer, batch_size):
    attack_critic_model_loop = load_critic_model(model_id)
    attack_gan_model_a, attack_gan_model_b = load_gan_model(model_id)
        
    # define optimization models
    optim_model, gen_latent_values = create_new_model_optim(target_model,
                                                                attack_critic_model_loop, 
                                                                attack_gan_model_a, 
                                                                attack_gan_model_b,)
    optim_model = freeze_not_optim_layers(optim_model)
    optim_model.compile(optimizer=optimizer,loss=loss)
    optim_model.fit(x, y, epochs=n_epochs, batch_size = batch_size)
    
    latent_reconstructed = gen_latent_values.predict(np.full((1, 1), 1))
    image_reconstructed = attack_gan_model_a.predict(latent_reconstructed)
    probs_reconstructed = target_model.predict(image_reconstructed)
    
    return optim_model, gen_latent_values, probs_reconstructed

In [ ]:
def compute_loss(target_probs, x, optim_model, _lambda):
    y_pred = optim_model.predict(x)
            # loss part 1
    loss_result_1 = np.mean(y_pred[1], axis=0) - target_probs
    loss_result_1 = np.abs(loss_result_1)
    loss_result_1 = np.mean(loss_result_1)/2
    loss_result_1 = lambda_*loss_result_1
            # loss part 2
    loss_result_2 = np.mean(y_pred[0], axis=0)
            # loss
    loss_result = loss_result_1 + loss_result_2
    
    return loss_result

In [ ]:
def optim_loop_repeated(target_probs, n_epochs, model_id, target_model_name, loss, optimizer, n=5, batch_size=32):

    target_model = load_model(target_model_name)
    
    x = np.full((500, 1), 1)
    y = [np.full((500, 1), 1), np.full((500, 5), 1)]
    
    image_reconstructed_list = []
    latent_reconstructed_list = []
    probs_reconstructed_list = []
    loss_list = []
    loss_1_list = []
    loss_2_list = []
    
    test = []

    for i in range(n) :
        
        optim_model, gen_latent_values, probs_reconstructed = init_and_fit_optim_model(x, y, target_model, n_epochs, model_id, loss, optimizer, batch_size)
        
        if np.argmax(probs_reconstructed)==np.argmax(target_probs):
            optim_model.fit(x, y, epochs=3*n_epochs, batch_size = batch_size)
            
            latent_reconstructed = gen_latent_values.predict(np.full((1, 1), 1))
            image_reconstructed = attack_gan_model_a.predict(latent_reconstructed)
            probs_reconstructed = target_model.predict(image_reconstructed)
            loss = compute_loss(target_probs, x, optim_model, _lambda)
            
            latent_reconstructed_list.append(latent_reconstructed)
            image_reconstructed_list.append(image_reconstructed)
            probs_reconstructed_list.append(probs_reconstructed)
            loss_list.append(loss)
    
    if len(loss_list)==0:
        return None, None, None, None, None, None
    else:
        return latent_reconstructed_list, loss_list, image_reconstructed_list, probs_reconstructed_list, optim_model, gen_latent_values

In [ ]:
optim_loop_repeated(target_probs, 1, model_id, target_model_name, loss, optimizer, n=2, batch_size=32)

## 3.2 Define saving process

In [ ]:
def return_results_df(): 
    if "results" not in os.listdir():
        os.mkdir("results")
    
    if "images" not in os.listdir("results"):
        os.mkdir("results/images")
    
    if "latent_vects" not in os.listdir("results"):
        os.mkdir("results/latent_vects")

    if "results.csv" not in os.listdir("results"):
        dict_results = ['model_id',
                             'n_epochs',
                             'target_model_name',
                             'latent_reconstructed',
                             'true_value',
                             'prediction',
                             'date',
                             'loss',
                             'image_reconstructed']

        pd_results = pd.DataFrame(columns = dict_results)
    else :
        pd_results = pd.read_csv("results/results.csv")
                        
    return pd_results

In [ ]:
def make_dict_of_results(target_probs, target_model_name, loss_list, 
                         latent_reconstructed_list, probs_reconstructed_list, 
                         image_reconstructed_list, num):
        target = np.argmax(target_probs)
        date = datetime.datetime.now().timestamp()
        evaluation_model = load_model(evaluation_model_name)

        loss = np.min(loss_list)
        best_position =  np.argmin(loss_list)
        best_latent_reconstructed = latent_reconstructed_list[best_position]
        # best_target_prediction = np.argmax(probs_reconstructed_list[best_position])
        best_image_reconstructed = image_reconstructed_list[best_position]
        best_target_prediction = np.argmax(evaluation_model.predict(best_image_reconstructed))
        
        best_latent_reconstructed_name = "latent_{}_{}".format(target, num)
        best_image_reconstructed_name = "image_{}_{}".format(target, num)

        # create the dict of the results
        result_dict = {'model_id': model_id,
                       'n_epochs': n_epochs,
                       'n_seeds': n_seeds,
                       'target_model_name': target_model_name,
                       'latent_reconstructed': best_latent_reconstructed_name,
                       'true_value': target,
                       'prediction_evaluation': best_target_prediction,
                       'date': date,
                       'loss': np.min(loss_list),
                       'image_reconstructed': best_image_reconstructed_name}
        return result_dict

In [ ]:
x_train, y_train, x_test, y_test = load_mnist_data("private")
img, label = pick_and_show_image(x_train, y_train)
img_rec = np.expand_dims(np.expand_dims(img,0),-1)
make_dict_of_results(target_probs, target_model_name, loss_list, [np.full((500, 1), 1)], [0,1,0,0,0,0], [img_rec], 1)

In [ ]:
def launch_experiment_and_save_results(target_probs,
                                       n_epochs,
                                       model_id,
                                       target_model_name,
                                       evaluation_model_name,
                                       loss,
                                       optimizer,
                                       n_seeds,
                                       batch_size=32):
    
    # launch the experiment
    latent_reconstructed_list, loss_list,\
    image_reconstructed_list, probs_reconstructed_list, optim_model, gen_latent_values = optim_loop_repeated(target_probs, 
                                                                             n_epochs,
                                                                             model_id,
                                                                             target_model_name,
                                                                             loss,
                                                                             optimizer,
                                                                             n_seeds,
                                                                             batch_size=32)
    if loss_list!=None:
        # create the dataframe of the results
        pd_results = return_results_df()
        num = len(pd_results)

        # build up objects to save
        result_dict = make_dict_of_results(target_probs,
                                           evaluation_model_name,
                                           loss_list,
                                           latent_reconstructed_list,
                                           probs_reconstructed_list,
                                           image_reconstructed_list,
                                           num)

        # save the csv file
        pd_results = pd_results.append(pd.Series(result_dict), ignore_index=True)
        pd_results.to_csv("results/results.csv", index=False)

        # save the latent vect
        np.save("results/latent_vects/{}".format(results_dict["latent_reconstructed"]), best_latent_reconstructed)

        # save the image
        np.save("results/images/{}".format(results_dict["image_reconstructed"]), best_image_reconstructed)

        return loss_list, probs_reconstructed_list, image_reconstructed_list, latent_reconstructed_list, optim_model, gen_latent_values
    else:
        return None, None, None, None, None, None

## 3.3 Settle parameters

In [ ]:
# models used
target_model_name = "model/target_model.h5"
evaluation_model_name = "model/evaluation_model.h5"
model_id = 13770

# optim parameters
lambda_ = 3000
n_epochs = 1
n_seeds = 2

# target
target_probs = np.array([0, 0, 1, 0, 0])

# Optim parameters
# TODO : mettre en paramètre le caractéristiques du programme d'optimisation
loss = loss_optim_fn(target_probs, lambda_)
optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True) # adam à tester
batch_size = 32

# target
target_probs = [0, 1, 0, 0, 0]

## 3.4 Launch the optim

In [ ]:
loss_list, probs_reconstructed_list, image_reconstructed_list, \
latent_reconstructed_list, optim_model, gen_latent_values = launch_experiment_and_save_results(target_probs,
                                                                                               n_epochs,
                                                                                               model_id,
                                                                                               target_model_name,
                                                                                               evaluation_model_name,
                                                                                               loss,
                                                                                               optimizer,
                                                                                               n_seeds,batch_size=32)

In [ ]:
plt.imshow(test[0][0,:,:,0])

In [ ]:
[np.argmax(x) for x in probs_reconstructed_list]

In [ ]:
plt.imshow(image_reconstructed_list[6][0,:,:,0])
plt.savefig("un.png")

In [ ]:
plt.imshow(test[1][0,:,:,0])

In [ ]:
latent_vect

In [ ]:
img = attack_gan_model_1.predict(latent_vect/np.max(latent_vect))
plt.imshow(img[0,:,:,0])

# Read the image

In [ ]:
result_num=0

In [ ]:
def write_title(result_num):
    pd_results = pd.read_csv("results/results.csv")
    truth = pd_results.loc[result_num].true_value
    pred = pd_results.loc[result_num].prediction
    title = "Value to find : {} / value found : {}".format(truth, pred)
    return title

In [ ]:
def plot_img(num):
    image_np = np.load("results/images/image_{}.npy".format(num))[0,:,:,0]
    plt.imshow(image_np)
    plt.title(write_title(num))

In [ ]:
plot_img(result_num)

In [ ]:
plot_img(2)

In [ ]:
pd_results = pd.read_csv("results/results.csv")
pd_results